In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce


In [2]:
# read all the worksheets form the data excel file
df = pd.read_excel('data/Muesli Project raw data - group 3.xlsx', header=1)
df1 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=1)
df2 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=2)
df3 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=3)

In [3]:
# check the df headers
print(df.columns)
print(df1.columns)
print(df2.columns)
print(df3.columns)

Index(['Index', 'Order ID', 'Order Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Origin Channel', 'Country/Region', 'City', 'State',
       'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product ID',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date', 'Customer Name'], dtype='object')
Index(['Row ID', 'Order ID', 'Order Date', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


In [10]:
# delete unneeded columns
df = df.drop(['Index','Origin Channel', 'Category', 'Sub-Category'], axis=1)
df1 = df1.drop(['Customer Name'], axis=1)
df2 = df2.drop(['Row ID', 'Order Date'], axis=1)

In [11]:
# check the number of rows and columns
print('df=',df.shape)
print('df1=',df1.shape)
print('df2=',df2.shape)
print('df3=',df3.shape)

df= (9994, 15)
df1= (333, 2)
df2= (5899, 3)
df3= (290, 3)


In [12]:
# check for duplicates
print('df=',df.duplicated().value_counts())
print('-' * 30)
print('df1=',df1.duplicated().value_counts())
print('-' * 30)
print('df2=',df2.duplicated().value_counts())
print('-' * 30)
print('df3=',df3.duplicated().value_counts())

df= False    9993
True        1
Name: count, dtype: int64
------------------------------
df1= False    333
Name: count, dtype: int64
------------------------------
df2= False    3003
True     2896
Name: count, dtype: int64
------------------------------
df3= False    204
True      86
Name: count, dtype: int64


In [13]:
# drop the duplicates
df_no_dups = df.drop_duplicates()
df2_no_dups = df2.drop_duplicates()
df3_no_dups = df3.drop_duplicates()

In [14]:
# check the number of rows and columns
print('df_no_dups=', df_no_dups.shape)
print('df1=', df1.shape)
print('df2_no_dups=', df2_no_dups.shape)
print('df3_no_dups=', df3_no_dups.shape)

df_no_dups= (9993, 15)
df1= (333, 2)
df2_no_dups= (3003, 3)
df3_no_dups= (204, 3)


In [16]:
# check the df headers and decide what to merge with what
print(df_no_dups.columns)
print(df1.columns)
print(df2_no_dups.columns)
print(df3_no_dups.columns)

Index(['Order ID', 'Order Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Country/Region', 'City', 'State', 'Postal Code', 'Region',
       'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date'], dtype='object')
Index(['Order ID', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


We have to calculate and validate the company assumptions:
1. Preparation process duration from order reception to shipping (2 days): 'Order Date' - 'Ready to Ship Day' (merge df and df3)
2. Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge df2 and df3
3. Efficacy of "Express Processing": the order ought to be on the truck on the ready to ship day
4. From "On Truck Scan" to "Arrival Scan Date"

...and get image of the whole delivery process duration: from 'Order Date' to 'Arrival Scan Date' (merge df and df2)



1. preparation process duration verification

In [17]:
# merge df and df3

df_prep_dur_ver = pd.merge(df_no_dups, df3_no_dups, on='Order ID', how='inner')

In [18]:
# verify the headers of the merged dataframe

df_prep_dur_ver.columns

Index(['Order ID', 'Order Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Country/Region', 'City', 'State', 'Postal Code', 'Region',
       'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Ready to Ship Date', 'Pickup Date'],
      dtype='object')

In [19]:
# delete unneeded columns

df_prep_dur_ver = df_prep_dur_ver.drop(['Ship Mode', 'Country/Region', 'City', 'State', 
                                        'Postal Code', 'Region', 'Product ID', 
                                        'Sales', 'Quantity', 'Discount', 'Profit'], axis=1)

In [20]:
# verify

df_prep_dur_ver.sample(10)

,Order ID,Order Date,Customer ID,Customer Name,Ready to Ship Date,Pickup Date
319,CA-2019-123176,2019-09-27,JG-15160,James Galang,2019-09-30,2019-10-02
18,US-2020-153948,2020-11-06,FM-14290,Frank Merwin,2020-11-06,2020-11-06
87,US-2019-118780,2019-09-10,PN-18775,Parhena Norris,2019-09-16,2019-09-18
390,CA-2020-155621,2020-11-08,KN-16450,Kean Nguyen,2020-11-16,2020-11-18
233,US-2020-155425,2020-11-10,AB-10600,Ann Blume,2020-11-11,2020-11-12
387,CA-2020-148985,2020-11-11,TB-21190,Thomas Brumley,2020-11-16,2020-11-18
88,CA-2019-130267,2019-09-19,SW-20245,Scot Wooten,2019-09-23,2019-09-25
244,CA-2020-154732,2020-11-05,AH-10195,Alan Haines,2020-11-09,2020-11-09
31,CA-2020-164028,2020-11-24,JL-15835,John Lee,2020-11-30,2020-12-02
33,CA-2019-142370,2019-09-19,TP-21130,Theone Pippenger,2019-09-23,2019-09-25


In [21]:
# check the type of data in the columns

df_prep_dur_ver.dtypes

Order ID                      object
Order Date            datetime64[ns]
Customer ID                   object
Customer Name                 object
Ready to Ship Date    datetime64[ns]
Pickup Date           datetime64[ns]
dtype: object

In [22]:
# calculate the preparation duration and print it in a new column

df_prep_dur_ver['prep duration'] = df_prep_dur_ver['Ready to Ship Date']-df_prep_dur_ver['Order Date']

In [23]:
# verify

df_prep_dur_ver.sample(10)

,Order ID,Order Date,Customer ID,Customer Name,Ready to Ship Date,Pickup Date,prep duration
93,CA-2020-121615,2020-11-03,DL-12925,Daniel Lacy,2020-11-09,2020-11-12,6 days
319,CA-2019-123176,2019-09-27,JG-15160,James Galang,2019-09-30,2019-10-02,3 days
166,CA-2020-150266,2020-11-25,RO-19780,Rose O'Brian,2020-11-30,2020-12-02,5 days
180,CA-2020-127782,2020-11-02,TH-21115,Thea Hudgings,2020-11-06,2020-11-09,4 days
37,US-2020-168116,2020-11-04,GT-14635,Grant Thornton,2020-11-04,2020-11-04,0 days
18,US-2020-153948,2020-11-06,FM-14290,Frank Merwin,2020-11-06,2020-11-06,0 days
74,CA-2019-165918,2019-09-10,BD-11770,Bryan Davis,2019-09-16,2019-09-18,6 days
152,CA-2020-119746,2020-11-23,CM-12385,Christopher Martinez,2020-11-27,2020-11-30,4 days
45,CA-2020-103380,2020-11-21,BF-11005,Barry Franz,2020-11-27,2020-11-30,6 days
355,US-2020-110576,2020-11-28,RB-19795,Ross Baird,2020-12-04,2020-12-07,6 days


In [24]:
# check the data type

df_prep_dur_ver['prep duration'].dtype

dtype('<m8[ns]')

what?...

dtype('<m8[ns]') indicates that the data type of the object is a datetime64, representing date and time information with nanosecond precision.

ok, now I get it!

In [28]:
# calculate the mean prep duration
mean_duration_days = df_prep_dur_ver['prep duration'].mean()

# Round the mean duration to the nearest days and hours
rounded_duration = mean_duration_days.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_days = rounded_duration.days
rounded_hours = rounded_duration.seconds // 3600  # Convert seconds to hours

print(f"{rounded_days} days {rounded_hours} hours")

4 days 4 hours


that's way more than the 2 days claimed preparation duration!

2. Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge df2 and df3


In [29]:
# merge df2 and df3

df_dur_readytoship_truck = pd.merge(df2_no_dups, df3_no_dups, on='Order ID', how='inner')

In [30]:
# verify the merged dataframe

df_dur_readytoship_truck.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date
52,CA-2019-132479,2019-09-27,Express,2019-09-26,2019-09-27
93,US-2020-122672,2020-11-12,Standard Processing,2020-11-09,2020-11-12
97,CA-2020-150959,2020-11-13,Express,2020-11-13,2020-11-13
6,CA-2019-106278,2019-09-06,Standard Processing,2019-09-05,2019-09-06
146,CA-2020-131254,2020-11-23,Express,2020-11-23,2020-11-23
102,CA-2020-152912,2020-11-13,Standard Processing,2020-11-12,2020-11-13
29,CA-2019-112893,2019-09-16,Standard Processing,2019-09-13,2019-09-16
168,CA-2020-156622,2020-11-27,Express,2020-11-26,2020-11-27
177,CA-2020-105130,2020-11-30,Standard Processing,2020-11-27,2020-11-30
28,CA-2019-153157,2019-09-16,Express,2019-09-16,2019-09-16


In [31]:
# calculate the duration of the process from "Ready to Ship" to "on Truck"  and print it in a new column

df_dur_readytoship_truck['ship to truck'] = df_dur_readytoship_truck['On Truck Scan Date']-df_dur_readytoship_truck['Ready to Ship Date']

In [32]:
# verify

df_dur_readytoship_truck.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
147,CA-2020-121818,2020-11-23,Express,2020-11-23,2020-11-23,0 days
200,US-2020-110576,2020-12-07,Standard Processing,2020-12-04,2020-12-07,3 days
156,CA-2020-120404,2020-11-25,Standard Processing,2020-11-24,2020-11-25,1 days
143,US-2020-132444,2020-11-23,Express,2020-11-23,2020-11-23,0 days
29,CA-2019-112893,2019-09-16,Standard Processing,2019-09-13,2019-09-16,3 days
87,CA-2020-142636,2020-11-12,Standard Processing,2020-11-09,2020-11-12,3 days
116,CA-2020-148985,2020-11-18,Standard Processing,2020-11-16,2020-11-18,2 days
49,CA-2019-145611,2019-09-25,Standard Processing,2019-09-24,2019-09-25,1 days
170,CA-2020-135307,2020-11-27,Express,2020-11-27,2020-11-27,0 days
137,US-2020-153633,2020-11-20,Standard Processing,2020-11-18,2020-11-20,2 days


In [33]:
df_dur_readytoship_truck['ship to truck'].unique()

<TimedeltaArray>
['1 days', '0 days', '3 days', '2 days']
Length: 4, dtype: timedelta64[ns]

In [34]:
# check the type of data in the columns

df_dur_readytoship_truck.dtypes

Order ID                       object
On Truck Scan Date     datetime64[ns]
Ship Mode                      object
Ready to Ship Date     datetime64[ns]
Pickup Date            datetime64[ns]
ship to truck         timedelta64[ns]
dtype: object

In [35]:
# filter for the standard processing

df_dur_readytoship_truck_standard = df_dur_readytoship_truck[df_dur_readytoship_truck['Ship Mode']== 'Standard Processing']

In [36]:
# verify

df_dur_readytoship_truck_standard.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
129,CA-2020-164364,2020-11-18,Standard Processing,2020-11-16,2020-11-18,2 days
55,CA-2019-123176,2019-10-02,Standard Processing,2019-09-30,2019-10-02,2 days
161,CA-2020-124674,2020-11-25,Standard Processing,2020-11-23,2020-11-25,2 days
13,CA-2019-149783,2019-09-11,Standard Processing,2019-09-09,2019-09-11,2 days
182,US-2020-109610,2020-12-02,Standard Processing,2020-11-30,2020-12-02,2 days
130,US-2020-132220,2020-11-18,Standard Processing,2020-11-16,2020-11-18,2 days
198,CA-2020-107125,2020-12-04,Standard Processing,2020-12-02,2020-12-04,2 days
18,CA-2019-169971,2019-09-11,Standard Processing,2019-09-09,2019-09-11,2 days
56,US-2019-134488,2019-10-02,Standard Processing,2019-10-01,2019-10-02,1 days
32,US-2019-118780,2019-09-18,Standard Processing,2019-09-16,2019-09-18,2 days


In [37]:
# calculate the mean prep duration for the standard processing
mean_to_truck_days_standard = df_dur_readytoship_truck_standard['ship to truck'].mean()

# Round the mean duration to the nearest days and hours
rounded_to_truck_dur_standard = mean_to_truck_days_standard.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_to_truck_days_standard = rounded_to_truck_dur_standard.days
rounded_to_truck_hours_standard = rounded_to_truck_dur_standard.seconds // 3600  # Convert seconds to hours

print(f"{rounded_to_truck_days_standard} days {rounded_to_truck_hours_standard} hours")

2 days 0 hours


...that's more than the claimed one day

3. Efficacy of "Express Processing": the order ought to be on the truck on the ready to ship day


In [38]:
# filter for the express processing

df_dur_readytoship_truck_xpress = df_dur_readytoship_truck[df_dur_readytoship_truck['Ship Mode']== 'Express']

In [39]:
# verify

df_dur_readytoship_truck_xpress.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
66,US-2020-145597,2020-11-06,Express,2020-11-05,2020-11-06,1 days
7,CA-2019-145240,2019-09-09,Express,2019-09-09,2019-09-09,0 days
41,CA-2019-158610,2019-09-23,Express,2019-09-23,2019-09-23,0 days
110,US-2020-118598,2020-11-16,Express,2020-11-16,2020-11-16,0 days
76,CA-2020-154732,2020-11-09,Express,2020-11-09,2020-11-09,0 days
147,CA-2020-121818,2020-11-23,Express,2020-11-23,2020-11-23,0 days
28,CA-2019-153157,2019-09-16,Express,2019-09-16,2019-09-16,0 days
148,CA-2020-111556,2020-11-23,Express,2020-11-23,2020-11-23,0 days
11,CA-2019-110023,2019-09-11,Express,2019-09-11,2019-09-11,0 days
149,CA-2020-157420,2020-11-23,Express,2020-11-23,2020-11-23,0 days


In [40]:
# calculate the mean prep duration for the express processing
mean_to_truck_days_xpress = df_dur_readytoship_truck_xpress['ship to truck'].mean()

# Round the mean duration to the nearest days and hours
rounded_to_truck_dur_xpress = mean_to_truck_days_xpress.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_to_truck_days_xpress = rounded_to_truck_dur_xpress.days
rounded_to_truck_hours_xpress = rounded_to_truck_dur_xpress.seconds // 3600  # Convert seconds to hours

print(f"{rounded_to_truck_days_xpress} days {rounded_to_truck_hours_xpress} hours")

0 days 10 hours


no comments

4a. From "On Truck Scan" to "Arrival Scan Date"


In [41]:
# merge df1 and df2

df_order_del_time = pd.merge(df2_no_dups, df1, on='Order ID', how='inner')

In [42]:
# verify

df_order_del_time.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Arrival Scan Date
264,CA-2019-112697,2019-12-23,Standard Processing,2019-12-27
221,CA-2019-164511,2019-11-27,Standard Processing,2019-11-29
309,CA-2020-122763,2020-03-20,Express,2020-03-23
237,US-2019-163881,2019-12-04,Standard Processing,2019-12-06
127,CA-2019-106075,2019-09-25,Standard Processing,2019-09-30
29,CA-2019-137729,2019-05-13,Standard Processing,2019-05-17
45,US-2019-117037,2019-05-20,Express,2019-05-27
164,CA-2019-149685,2019-10-16,Standard Processing,2019-10-21
12,CA-2019-161816,2019-05-03,Express,2019-05-07
249,CA-2019-142594,2019-12-11,Standard Processing,2019-12-13


In [43]:
df_order_del_time['delivery time']=df_order_del_time['Arrival Scan Date']- df_order_del_time['On Truck Scan Date']

In [44]:
# verify
df_order_del_time.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Arrival Scan Date,delivery time
172,CA-2019-169887,2019-10-23,Express,2019-10-28,5 days
189,US-2019-142251,2019-11-06,Standard Processing,2019-11-11,5 days
173,CA-2019-147256,2019-10-23,Standard Processing,2019-10-28,5 days
111,CA-2019-133123,2019-09-13,Standard Processing,2019-09-16,3 days
223,CA-2019-100041,2019-11-27,Standard Processing,2019-12-02,5 days
147,CA-2019-143406,2019-10-02,Standard Processing,2019-10-07,5 days
281,CA-2020-111738,2020-01-08,Standard Processing,2020-01-10,2 days
288,CA-2020-131954,2020-01-29,Standard Processing,2020-02-03,5 days
301,CA-2020-124114,2020-03-02,Express,2020-03-06,4 days
263,US-2019-169369,2019-12-20,Standard Processing,2019-12-24,4 days


In [45]:
# calculate the mean prep duration for the standard processing
mean_order_del_time = df_order_del_time['delivery time'].mean()

# Round the mean duration to the nearest days and hours
rounded_mean_del_time = mean_order_del_time.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_mean_del_days = rounded_mean_del_time.days
rounded_mean_del_hours = rounded_mean_del_time.seconds // 3600  # Convert seconds to hours

print(f"{rounded_mean_del_days} days {rounded_mean_del_hours} hours")

4 days 14 hours


the shipping company claim is three days, oups!

4b. From "Pickup Date" to "Arrival Scan Date"

In [46]:
# merge df1 and df3

df_order_del_time2 = pd.merge(df1, df3_no_dups, on='Order ID', how='inner')

In [47]:
# verify

df_order_del_time2.sample(10)

,Order ID,Arrival Scan Date,Ready to Ship Date,Pickup Date
11,US-2019-118780,2019-09-23,2019-09-16,2019-09-18
7,CA-2019-110023,2019-09-16,2019-09-11,2019-09-11
18,CA-2019-145611,2019-09-30,2019-09-24,2019-09-25
2,CA-2019-106278,2019-09-10,2019-09-05,2019-09-06
13,CA-2019-145625,2019-09-23,2019-09-17,2019-09-18
10,CA-2019-121223,2019-09-19,2019-09-13,2019-09-16
14,CA-2019-130225,2019-09-23,2019-09-17,2019-09-18
1,CA-2019-129847,2019-09-10,2019-09-04,2019-09-04
15,CA-2019-102932,2019-09-30,2019-09-23,2019-09-25
4,CA-2019-123120,2019-09-16,2019-09-09,2019-09-11


In [48]:
df_order_del_time2['delivery time2']=df_order_del_time2['Arrival Scan Date']- df_order_del_time2['Pickup Date']

In [49]:
# verify

df_order_del_time2.sample(10)

,Order ID,Arrival Scan Date,Ready to Ship Date,Pickup Date,delivery time2
16,CA-2019-168557,2019-09-30,2019-09-23,2019-09-25,5 days
3,CA-2019-158099,2019-09-11,2019-09-05,2019-09-06,5 days
12,CA-2019-146206,2019-09-23,2019-09-16,2019-09-18,5 days
13,CA-2019-145625,2019-09-23,2019-09-17,2019-09-18,5 days
10,CA-2019-121223,2019-09-19,2019-09-13,2019-09-16,3 days
0,CA-2019-129630,2019-09-09,2019-09-04,2019-09-04,5 days
14,CA-2019-130225,2019-09-23,2019-09-17,2019-09-18,5 days
17,CA-2019-100244,2019-09-30,2019-09-24,2019-09-25,5 days
20,CA-2019-141551,2019-10-07,2019-09-30,2019-10-02,5 days
19,CA-2019-166772,2019-09-30,2019-09-24,2019-09-25,5 days


In [50]:
# calculate the mean prep duration for the standard processing
mean_order_del_time2 = df_order_del_time2['delivery time2'].mean()

# Round the mean duration to the nearest days and hours
rounded_mean_del_time2 = mean_order_del_time2.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_mean_del2_days = rounded_mean_del_time2.days
rounded_mean_del2_hours = rounded_mean_del_time2.seconds // 3600  # Convert seconds to hours

print(f"{rounded_mean_del2_days} days {rounded_mean_del2_hours} hours")

4 days 16 hours



...and get image of the whole delivery process duration: from 'Order Date' to 'Arrival Scan Date' (merge df and df2)

In [69]:
# merge df and df1

df_order_del_total = pd.merge(df1, df_no_dups, on='Order ID', how='inner')

In [70]:
# verify

df_order_del_total.sample(10)

,Order ID,Arrival Scan Date,Order Date,Ship Mode,Customer ID,Customer Name,Country/Region,City,State,Postal Code,Region,Product ID,Sales,Quantity,Discount,Profit
70,CA-2019-169166,2019-05-20,2019-05-09,Standard Class,SS-20590,Sonia Sunley,United States,Seattle,Washington,98115.0,West,TEC-AC-10000991,93.980,2,0.0,13.1572
162,US-2019-105578,2019-06-10,2019-05-30,Standard Class,MY-17380,Maribeth Yedwab,United States,Parker,Colorado,80134.0,West,FUR-CH-10001215,801.568,2,0.2,50.0980
417,CA-2019-134110,2019-11-25,2019-11-17,First Class,BG-11035,Barry Gonzalez,United States,The Colony,Texas,75056.0,Central,TEC-PH-10002350,67.176,3,0.2,6.7176
292,CA-2019-143406,2019-10-07,2019-09-26,Standard Class,LR-17035,Lisa Ryan,United States,Houston,Texas,77041.0,Central,FUR-CH-10000513,454.965,5,0.3,-136.4895
43,CA-2019-144792,2019-05-13,2019-04-30,Standard Class,KD-16615,Ken Dana,United States,Scottsdale,Arizona,85254.0,West,FUR-FU-10002759,111.888,7,0.2,22.3776
270,CA-2019-166653,2019-10-01,2019-09-22,Second Class,NP-18685,Nora Pelletier,United States,Parma,Ohio,44134.0,East,OFF-EN-10003055,63.968,2,0.2,19.9900
56,CA-2019-106530,2019-05-13,2019-05-08,First Class,CL-12565,Clay Ludtke,United States,Cleveland,Ohio,44105.0,East,OFF-ST-10002011,1006.056,3,0.2,88.0299
240,CA-2019-105732,2019-09-25,2019-09-13,Standard Class,AG-10270,Alejandro Grove,United States,Omaha,Nebraska,68104.0,Central,OFF-ST-10004340,373.080,6,0.0,100.7316
159,US-2019-105578,2019-06-10,2019-05-30,Standard Class,MY-17380,Maribeth Yedwab,United States,Parker,Colorado,80134.0,West,OFF-BI-10001670,22.620,2,0.7,-15.0800
280,US-2019-155404,2019-10-07,2019-09-22,Standard Class,AS-10630,Ann Steele,United States,Louisville,Kentucky,40214.0,South,FUR-FU-10004586,13.280,2,0.0,6.3744


In [71]:
df_order_del_total = df_order_del_total.drop(['Ship Mode', 'Country/Region', 'City', 'State', 
                                        'Postal Code', 'Region', 'Product ID', 
                                        'Sales', 'Quantity', 'Discount', 'Profit'], axis=1)

In [72]:
# verify

df_order_del_total.sample(10)

,Order ID,Arrival Scan Date,Order Date,Customer ID,Customer Name
415,CA-2019-149482,2019-11-25,2019-11-14,RR-19315,Ralph Ritter
200,CA-2019-123120,2019-09-16,2019-09-04,CV-12295,Christina VanderZanden
508,CA-2019-131289,2019-12-23,2019-12-08,SP-20620,Stefania Perrino
326,CA-2019-156503,2019-10-28,2019-10-14,NC-18415,Nathan Cano
655,CA-2020-134845,2020-04-30,2020-04-17,SR-20425,Sharelle Roach
280,US-2019-155404,2019-10-07,2019-09-22,AS-10630,Ann Steele
257,CA-2019-133935,2019-09-30,2019-09-18,JW-15220,Jane Waco
545,CA-2020-111738,2020-01-10,2020-01-03,CM-12385,Christopher Martinez
495,CA-2019-143154,2019-12-16,2019-12-02,AS-10225,Alan Schoenberger
147,CA-2019-153682,2019-06-06,2019-05-30,BG-11695,Brooke Gillingham


In [73]:
df_order_del_total['total delivery time']=df_order_del_total['Arrival Scan Date']- df_order_del_total['Order Date']

In [74]:
# verify

df_order_del_total.sample(10)

,Order ID,Arrival Scan Date,Order Date,Customer ID,Customer Name,total delivery time
48,CA-2019-104311,2019-05-13,2019-05-02,AS-10090,Adam Shillingsburg,11 days
379,CA-2019-161669,2019-11-18,2019-11-07,EM-14095,Eudokia Martin,11 days
168,US-2019-144211,2019-09-09,2019-08-28,CS-12130,Chad Sievert,12 days
116,CA-2019-138282,2019-05-31,2019-05-19,AH-10690,Anna Häberlin,12 days
557,CA-2020-113278,2020-01-27,2020-01-14,HR-14770,Hallie Redmond,13 days
620,CA-2020-140949,2020-03-30,2020-03-17,DB-13405,Denny Blanton,13 days
412,CA-2019-116722,2019-11-25,2019-11-11,LP-17080,Liz Pelletier,14 days
110,CA-2019-100510,2019-05-28,2019-05-12,HM-14860,Harry Marie,16 days
614,CA-2020-122763,2020-03-23,2020-03-20,HG-14845,Harry Greene,3 days
282,CA-2019-149979,2019-10-07,2019-09-23,RA-19915,Russell Applegate,14 days


In [75]:
# calculate the mean prep duration for the standard processing
mean_order_del_total = df_order_del_total['total delivery time'].mean()

# Round the mean duration to the nearest days and hours
rounded_mean_del_total = mean_order_del_total.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_mean_del_total_days = rounded_mean_del_total.days
rounded_mean_del_total_hours = rounded_mean_del_total.seconds // 3600  # Convert seconds to hours

print(f"{rounded_mean_del_total_days} days {rounded_mean_del_total_hours} hours")

10 days 21 hours


In [ ]:
#merge the dataframes
merged_df = pd.merge(df, df1, on='Order ID', how='inner')
merged_df1 = pd.merge(merged_df, df2_red_no_dups, on='Order ID', how='inner')
merged_df2 = pd.merge(merged_df1, df3_no_dups, on='Order ID', how='inner')

In [ ]:
# merge the dataframes with the 'reduce' function
dfs = [df, df1, df2_red_no_dups, df3_no_dups]

merged_df_red = reduce(lambda left, right: pd.merge(left, right, on='Order ID', how='inner'), dfs)

merged_df_red.shape